In [2]:
import pandas as pd

# -- Selenium --
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from time import sleep
from selenium.webdriver.common.by import By


# -- API --
import re
import json
import requests
from tqdm import tqdm

# Get today
from datetime import date

# Check time
from tqdm import tqdm

In [3]:
data = pd.read_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/crawl_web_RA/20220822_shopee/shopee_20220829/link_shop_shopee_recrawl.csv')

In [5]:
data.columns

Index(['shop_url'], dtype='object')

In [6]:
product_links = data.shop_url

print(len(product_links))

2694


In [253]:
def get_location(driver):
    check_xpath_location = driver.find_elements(by=By.XPATH, value='//div[@class="OktMMO"]')
    for xpath in check_xpath_location:
        if('Gửi từ' in xpath.text):
            location = xpath.text.split('\n')[1]

    return location

def get_cmt_shop(driver):
    cmt_total = ''

    xpath_cmt_shop = driver.find_elements(by=By.XPATH, value='//div[@class="_2w33Zs"]')
    for xpath in xpath_cmt_shop:
        cmt_total = cmt_total + xpath.text

    return cmt_total

def get_categorys(driver):
    xpath_cat = driver.find_elements(by=By.XPATH, value='//a[@class="CyVtI7 _2yC5g9"]')
    total_cat = ''
    for xpath in xpath_cat:
        total_cat = total_cat + xpath.text + ' -> '

    return(total_cat[:-4])

In [ ]:
def login_shopee(driver):
    url_log_in = 'https://shopee.vn/buyer/login?next=https%3A%2F%2Fshopee.vn%2F'
    driver.get(url_log_in)

    _username = '_username'
    _password = '_password'

    username_field = driver.find_element(by=By.NAME, value='loginKey')
    username_field.send_keys(_username)

    password_field = driver.find_element(by=By.NAME, value='password')
    password_field.send_keys(_password)

    sleep(0.5)

    login_button = driver.find_element(by=By.XPATH, value='//button[@class="wyhvVD _1EApiB hq6WM5 L-VL8Q cepDQ1 _7w24N1"]')
    login_button.click()


    sleep(3)

In [255]:
def startCrawl(product_links, list_product):
    driver = webdriver.Chrome(executable_path=r"C:/Users/Admin/OneDrive_Quan/Máy tính/crawl_web_RA/chromedriver")

    # Login tk shopee
    url_log_in = 'https://shopee.vn/buyer/login?next=https%3A%2F%2Fshopee.vn%2F'
    driver.get(url_log_in)

    _username = '_username'
    _password = '_password'

    username_field = driver.find_element(by=By.NAME, value='loginKey')
    username_field.send_keys(_username)

    password_field = driver.find_element(by=By.NAME, value='password')
    password_field.send_keys(_password)

    sleep(0.5)

    login_button = driver.find_element(by=By.XPATH, value='//button[@class="wyhvVD _1EApiB hq6WM5 L-VL8Q cepDQ1 _7w24N1"]')
    login_button.click()


    sleep(3)
    # End

    for product_link in product_links:
        Product_Infos = {}

        driver.get(product_link)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(0.75)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(0.25)

        if len(driver.find_elements(by=By.NAME, value='loginKey')) != 0:
            username_field = driver.find_element(by=By.NAME, value='loginKey')
            username_field.send_keys(_username)

            password_field = driver.find_element(by=By.NAME, value='password')
            password_field.send_keys(_password)

            sleep(0.5)

            login_button = driver.find_element(by=By.XPATH, value='//button[@class="wyhvVD _1EApiB hq6WM5 L-VL8Q cepDQ1 _7w24N1"]')
            login_button.click()

            sleep(1)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            sleep(0.25)


        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(0.25)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(2)

        try:
            product_id, shop_id = product_link.split('?sp_atk')[0].split('i.')[1].split('.')
            Product_Infos['ProductIDs'] = product_id
            Product_Infos['ShopIDs'] = shop_id
        except:
            pass
        dict_xpath = {
                'Solds': '//div[@class="HmRxgn"]',
                'Names': '//div[@class="_2rQP1z"]/span',
                'LastCrawlSellPrices': '//div[@class="_2Shl1j"]',
                'RegularPrices': '//div[@class="_2yjfFH"]',
                'RatingScores': '//div[@class="_3y5XOB _14izon"]',
                'RatingCounts': '//div[@class="_3y5XOB"]',
                'Descriptions': '//p[@class="_2jrvqA"]'
        }
        for key, value in dict_xpath.items():
            try:
                Product_Infos[key] = driver.find_element(by=By.XPATH, value=value).text
            except:
                continue

        Product_Infos['ProductLinks'] = product_link
        Product_Infos['LastCrawlDates'] = str(date.today())

        try:
            Product_Infos['ShopReplyComments'] = get_cmt_shop(driver)
        except:
            pass

        try:
            Product_Infos['Categorys'] = get_categorys(driver)
        except:
            pass
        try:
            Product_Infos['Locations'] = get_location(driver)
        except:
            pass


        list_product.append(Product_Infos)
        print(len(list_product))
        sleep(0.5)



In [257]:
import threading
import numpy as np

list_threads = []
list_product = []
num_thread = 3
list_partitions = [partition for partition in np.array_split(product_links, num_thread)]
for partition_list in list_partitions:
    list_threads.append(threading.Thread(target=startCrawl, args=(partition_list, list_product)))

In [ ]:
for thread in list_threads:
    thread.start()  
    time.sleep(5)       

In [ ]:
for thread in list_threads:
    thread.join()  
       

In [260]:
len(list_product)

7656

In [262]:
pd.DataFrame(list_product).to_csv('shopee_product_info.csv', index=False)